In [1]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from datasets import load_from_disk, load_dataset
import wandb
import pandas as pd
import os
import yaml
import argparse
from transformers.trainer_callback import EarlyStoppingCallback
import numpy as np
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from datasets import Dataset, DatasetDict
from src.helper_functions import Config, prepare_text

/home/james/CodingProjects/ExplainPetBERT/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Import yaml file
with open("../configs/train_default.yaml") as f:
    args = yaml.safe_load(f)

config_type = "all_text"
# Update default args with chosen config
if config_type != "default":
    with open("../configs/train_configs.yaml") as f:
        yaml_configs = yaml.safe_load_all(f)
        yaml_args = next(conf for conf in yaml_configs if conf["config"] == config_type)
    args.update(yaml_args)
    print(f"Updating with:\n{yaml_args}\n")
print(f"\n{args}\n")

# Dataset
di = Config("../configs/dataset_info.yaml")
dataset = load_dataset(
    args["ds_name"],
    download_mode="force_redownload",
)
dataset = prepare_text(
    dataset=dataset,
    di=di,
    version=args["version"],
)

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(
    args["model_base"],
    num_labels=2,
)
tokenizer = AutoTokenizer.from_pretrained(args["model_base"])


# Tokenize the dataset
def encode(examples):
    return {
        "label": np.array([examples["labels"]]),
        **tokenizer(
            examples["text"],
            # truncation=True,
            # padding="max_length",
            # max_length=args["max_length"],
        ),
    }


dataset = dataset.map(encode)  # , load_from_cache_file=True)

Updating with:
{'config': 'all_text', 'fast_dev_run': False, 'batch_size': 32, 'ds_name': 'james-burton/vet_month_1_all_text', 'version': 'all_text'}


{'config': 'all_text', 'fast_dev_run': False, 'do_train': True, 'do_predict': True, 'tags': ['bert', '1 month'], 'batch_size': 32, 'model_base': 'bert-base-uncased', 'output_root': 'models/', 'num_epochs': 50, 'early_stopping_patience': 3, 'grad_accumulation_steps': 1, 'seed': 42, 'logging_steps': 10, 'lr_scheduler': 'linear', 'warmup_ratio': 0, 'device': 'cuda', 'num_workers': 1, 'resume_from_checkpoint': False, 'predict_batch_size': 16, 'save_total_limit': 1, 'pytorch2.0': True, 'max_length': 512, 'ds_name': 'james-burton/vet_month_1_all_text', 'version': 'all_text'}



Map: 100%|██████████| 1272/1272 [00:00<00:00, 3386.87 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1272/1272 [00:00<00:00, 1312.55 examples/s]


In [8]:
sum([len(ids) > 512 for ids in dataset["train"]["input_ids"]]) / len(
    dataset["train"]["input_ids"]
)

0.055925617540938104